In [1]:
#Librerías
import ee #earthengine
import geemap #Alternativa a GEE paquete
import matplotlib.pyplot as plt #Generación de gráficos
import pandas as pd
import numpy as np
import os

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map()

In [21]:
Map

Map(bottom=32744.20001220703, center=[1.778400830827573, -76.38576520736781], controls=(WidgetControl(options=…

In [5]:
AOI = 'D:/DOCS/PATIA/Sph.AltoPatia/AltoPatia.shp'

In [6]:
AOI = geemap.shp_to_ee(AOI)

In [7]:
Map.addLayer(AOI, {}, 'AOI')

# **Temperature Condition Index (TCI)**

<p>Land surface temperature (LST) derived from thermal radiance bands is a good indicator of the energy balance of the Earth’s surface, because temperatures can rise quickly under water stress. The TCI is an initial indicator of water stress and drought. It is calculated using the following formula <a href="https://un-spider.org/advisory-support/recommended-practices/recommended-practice-agriculture-drought-monitoring/in-detail"> (UN, 2017) </a>. </p><br>

$TCI_{j} = \frac{(TCI_{j} - TCI_{min})} {(TCI_{max} - TCI_{min})} × 100$

$TCI_{max}$ and $TCI_{min}$ are the maximum and minimum $TCI$ values in a multi-year dataset. The $j$ is the $TCI$ value for the current month.

# **<p style="background-color:Tomato;">INGRESO DE DATOS</p>**

In [8]:
fechaI = "2001-01-01" # yyyy-mm-dd --> Ingresar fecha inicial
fechaF = "2021-12-31" # yyyy-mm-dd --> Ingresar fecha final

In [9]:
Anios = ee.List.sequence(2001, 2021) # A partir de la fecha registrada en la celda anterior 
                              # incluir el rango de años correspondiente --> Año inicial, Año final + 1
Anios

In [10]:
Meses = ee.List.sequence(1,12)
Meses

In [11]:
MOD11A2_006 = ee.ImageCollection('MODIS/006/MOD11A2').filterBounds(AOI).filterDate(fechaI,fechaF)

# **<p style="background-color:Tomato;">PROCESAMIENTO DE DATOS</p>**

In [12]:
# Valor mínimo y máximo de temperaturas superficiales terrestres
minLST = MOD11A2_006.select('LST_Day_1km').min()
maxLST = MOD11A2_006.select('LST_Day_1km').max()

In [13]:
Lw = ee.List([])
for i in range(2001,2022):
    for j in range(1,13):
        w = MOD11A2_006.select('LST_Day_1km').filter(ee.Filter.calendarRange(ee.Number(i), ee.Number(i), 'year')).filter(ee.Filter.calendarRange(ee.Number(j), ee.Number(j), 'month')).mean()
        w = w.set('year', i)
        w = w.set('month', j)
        w = w.set('system:time_start', ee.Date.fromYMD(i, j, 1))
        Lw = Lw.add(ee.Image(w))
w = ee.ImageCollection.fromImages(Lw)

In [14]:
def TCIf(imgCol):
    TCI = (imgCol.subtract(minLST)).divide(maxLST.subtract(minLST)).clip(AOI)
    return TCI

In [15]:
TCI = w.map(TCIf)

In [16]:
vis =  ['d7191c', 'fdae61', 'ffffc0', 'a6d96a', '1a9641']
visParams = {'min': -1, 'max': 1, 'palette': vis}
Map.addLayer(TCI, visParams, 'TCI')

In [17]:
#Criterios de clasificacion
def ClasificarImgC(ImgCol):
    image02 = ee.Image(ImgCol.lt(0.1).And(ImgCol.gte(-1)))
    image04 = ee.Image(((ImgCol.gte(0.1)).And(ImgCol.lt(0.2))).multiply(2))
    image06 = ((ImgCol.gte(0.2)).And(ImgCol.lt(0.3))).multiply(3)
    image08 = ((ImgCol.gte(0.3)).And(ImgCol.lt(0.4))).multiply(4)
    image10 = (ImgCol.gte(0.4)).multiply(5)
    Drought_Index = (image02.add(image04).add(image06).add(image08).add(image10))
    Drought_Index = Drought_Index.float()
    return Drought_Index

In [18]:
IndiceSequia = TCI.map(ClasificarImgC)

In [19]:
vis2 =  ['d7191c', 'fdae61', 'ffffc0', 'a6d96a', '1a9641'] #Extreme (rojo),Severe (anaranjado), Moderate (amarillo), Mild (verde claro) and No Drought (verde oscuro)
visParams2 = {'min': 1, 'max': 5, 'palette': vis2}
Map.addLayer(IndiceSequia, visParams2, 'IndiceSequia')

In [20]:
# Indices de sequía

# 'Extreme' = 1
# 'Severe' = 2
# 'Moderate' = 3
# 'Mild' = 4
# 'No Drought' = 5